In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import pickle
import tensorflow as tf

from nncf import NNCFConfig
from nncf.tensorflow import create_compressed_model, create_compression_callbacks, register_default_init_args

Import dataset pickle file

In [ ]:
def get_pickle(pickle_dir, file_name):
    pickle_dir = os.path.join(os.getcwd(), 'datasets/fsc22/Pickle Files/' + pickle_dir)
    fold_dir = os.path.join(pickle_dir, file_name)
    infile = open(fold_dir, 'rb')
    fold = pickle.load(infile)
    infile.close()

    return fold

In [ ]:
with tf.device('/CPU:0'):
    pickle_dir = 'aug_ts_ps_mixed_features_5_20_5fold' # change pickle file directory name accordingly
    
    spect_folds = []
    train_spects = []
    
    for fold in range(5):
        spects_fold = get_pickle(pickle_dir, pickle_dir + '_fold' + str(fold+1))
        train_spects.extend(spects_fold)
        spect_folds.append(spects_fold)

    print(f'len train_spects: {len(train_spects)}')

    train_features_df = pd.DataFrame(train_spects, columns=['feature', 'class'])

    del train_spects

    gc.collect()
    
    print(f'len spect_folds: {len(spect_folds)}') # num folds
    print(f'len spect_folds[0]: {len(spect_folds[0])}') # samples in a fold
    print(f'len spect_folds[0][0]: {len(spect_folds[0][0])}') # elements in a sample - should be 2
    print(f'shape spect_folds[0][0][0]: {np.shape(spect_folds[0][0][0])}') # spectrogram shape

In [ ]:
with tf.device('/CPU:0'):
    IMG_SIZE = (spect_folds[0][0][0].shape[0], spect_folds[0][0][0].shape[1])
    IMG_SHAPE = IMG_SIZE + (3,)
    print(IMG_SHAPE)
    input_shape = IMG_SHAPE

    num_labels = 26 # Should be changed

Import base model

In [ ]:
with tf.device('/CPU:0'):
    valid_model_type = False
    model_type = None

    while not valid_model_type:
        model_type = input(
            "Choose Model Type:\n"
            " 1. AlexNet\n"
            " 2. DenseNet121\n"
            " 3. EfficientNetV2B0\n"
            " 4. InceptionV3\n"
            " 5. MobileNetV3Small\n"
            " 6. ResNet50V2\n"
            " 7. SqueezeNet\n :")

        if model_type == '1':
            model_type = 'AlexNet'
            valid_model_type = True
        elif model_type =='2':
            model_type = 'DenseNet121'
            valid_model_type = True
        elif model_type =='3':
            model_type = 'EfficientNetV2B0'
            valid_model_type = True
        elif model_type =='4':
            model_type = 'InceptionV3'
            valid_model_type = True
        elif model_type =='5':
            model_type = 'MobileNetV3-Small'
            valid_model_type = True
        elif model_type =='6':
            model_type = 'ResNet50V2'
            valid_model_type = True
        elif model_type =='7':
            model_type = 'SqueezeNet'
            valid_model_type = True

In [ ]:
with tf.device('/CPU:0'):
    models_dir = os.path.join(os.getcwd(), "models")
    model_dir = os.path.join(models_dir, model_type)

    # Example - 
    # model_name = 'EfficientNet_aug5_mix_5fold'
    # base_model_save_path = 'models/EfficientNetV2B0/EfficientNet_aug5_mix_5fold/EfficientNet_aug5_mix_5fold_weight_pruned_0_95_fold1.h5'
    # fold_no = 1
    
    model_name = input('Enter model name: ')
    base_model_save_path = input('Enter base model relative save path: ')
    fold_no = int(input('Enter the fold which the base model was validated: '))

    model_save_dir = os.path.join(model_dir, model_name)
    
    base_model = tf.keras.models.load_model(base_model_save_path)
    base_model.summary()

Create train and validation data

In [ ]:
def get_train_vaild_data(spects, fold):
    train_spects = []
    valid_spects = None
    
    for i in range(5):
        if i + 1 != fold:
            train_spects.extend(spects[i])
        else:
            valid_spects = spects[i]
    
    train_df = pd.DataFrame(train_spects, columns=['feature', 'class'])
    valid_df = pd.DataFrame(valid_spects, columns=['feature', 'class'])

    del train_spects
    del valid_spects

    gc.collect()

    X_train_cv = np.array(train_df['feature'].tolist())
    y_train_cv = np.array(train_df['class'].tolist())
    
    X_valid_cv = np.array(valid_df['feature'].tolist())
    y_valid_cv = np.array(valid_df['class'].tolist())
    
    return X_train_cv, X_valid_cv, y_train_cv, y_valid_cv

In [ ]:
X_train_cv, X_valid_cv, y_train_cv, y_valid_cv = get_train_vaild_data(spect_folds, fold_no)

print(f'X_train_cv shape: {np.shape(X_train_cv)}')
print(f'y_train_cv shape: {np.shape(y_train_cv)}')
print(f'X_valid_cv shape: {np.shape(X_valid_cv)}')
print(f'y_valid_cv shape: {np.shape(y_valid_cv)}')

Create base model

In [ ]:
# Change according to the hyperparameters of the base model

lr = 0.07263866958
best_optimizer = 'sgd'
batch_size = 16

optimizer = tf.keras.optimizers.Adam(learning_rate=lr) if best_optimizer=='adam' else tf.keras.optimizers.SGD(learning_rate=lr)
base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

base_model_loss, base_model_accuracy = base_model.evaluate(x=X_valid_cv,y=y_valid_cv)

In [ ]:
sample_size = [batch_size] + list(IMG_SHAPE)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_cv, y_train_cv))

### Filter pruning

Create filter pruning configuration

In [ ]:
pruning_target = float(input('Enter pruning target: ')) # Example - 0.8
pruning_steps = 15

nncf_config_dict = {
    "input_info": {"sample_size": sample_size}, # input shape required for model tracing
    "compression": [
        {
            "algorithm": "filter_pruning",
            "pruning_init": 0.0,
            "params": {
                "pruning_target": pruning_target,
                "pruning_steps": pruning_steps
            }
        },
    ]
}
nncf_config = NNCFConfig.from_dict(nncf_config_dict)
nncf_config = register_default_init_args(nncf_config, train_dataset, batch_size=batch_size) # train_dataset is an instance of tf.data.Dataset

Add filter pruning mask to model

In [ ]:
# model instance of the tensorflow.keras.Model
compression_ctrl, filter_pruned_model = create_compressed_model(base_model, nncf_config)

Filter prune the model

In [ ]:
# fine-tuning preparations, e.g. dataset, loss, optimizer setup, etc.

# create compression callbacks to control pruning parameters and dump compression statistics
# all the setting are being taked from compression_ctrl, i.e. from NNCF config
compression_callbacks = create_compression_callbacks(compression_ctrl, log_dir=model_save_dir + "/compression_log")

epochs = 5

filter_pruned_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# tune quantized model for 25 epochs as the baseline
filter_pruned_model.fit(X_train_cv, y_train_cv, batch_size=batch_size, epochs=epochs, validation_data=(X_valid_cv, y_valid_cv), callbacks=compression_callbacks)

In [ ]:
with tf.device('/CPU:0'):
    pruned_model_loss, pruned_model_accuracy = filter_pruned_model.evaluate(x=X_valid_cv,y=y_valid_cv)

    print('Baseline test accuracy:', base_model_accuracy)
    print('Pruned test accuracy:', pruned_model_accuracy)

Save filter pruned model with mask (as Frozen Graph .pb)

In [ ]:
pruned_model_name = base_model_save_path.split('\\')[-1].split('.')[0].replace(f'_fold{fold_no}', '') + "_filter_pruned_" + str(pruning_target).replace('.', '_') + "_fold" + str(fold_no) + ".pb"

filter_pruned_model_save_path = os.path.join(model_save_dir, pruned_model_name)

compression_ctrl.export_model(filter_pruned_model_save_path) #export to Frozen Graph

In [ ]:
import openvino as ov

# filter_pruned_model_save_path = 'models/EfficientNetV2B0/EfficientNet_aug5_mix_5fold/EfficientNet_aug5_mix_5fold_weight_pruned_0_95_filter_pruned_0_8_new_2_fold1.pb'

ov_model = ov.convert_model(filter_pruned_model_save_path)

# save model to OpenVINO IR for later use
ov.save_model(ov_model, filter_pruned_model_save_path.replace('.pb', '.xml'))

Save the filter prune model without mask as OpenVINO IR model

Change working directory to model directory

In [ ]:
%cd models/InceptionV3/Inception_aug5_mix_5fold

Save model without filter pruning mask

Enter the saved filter pruned model (.pb) as the --input_model argument

In [ ]:
import subprocess

# Your Windows command
command = 'mo --input_model Inception_aug5_mix_5fold_weight_pruned_0_8_filter_pruned_0_8_fold1.pb --transform=Pruning'

# Use subprocess.run() to execute the command
try:
    subprocess.run(command, shell=True, check=True)
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")

%cd ../../..